In [1]:
from prophet import Prophet
from sklearn.metrics import root_mean_squared_error
import pandas as pd
import numpy as np

#### Prophet -> WRMSE =

In [2]:
df = pd.read_excel('../Input/DadosCompeticao.xlsx')

In [3]:
wrmse = lambda rmse: (rmse[:11] * (1 / 11)).sum()

#### Pred and Forecast

In [10]:
def prophet(df, forecast_horizon=12):
    forecasts = {}
    scores = {}

    for col in df.columns:
        print(f"==== Treinando {col} ====")

        # Preparando a série no formato esperado pelo Prophet
        series = df[col].dropna()
        ts = pd.DataFrame({
            'ds': pd.date_range(start='2000-01-01', periods=len(series), freq='M'),
            'y': series.values
        })

        model = Prophet()
        model.fit(ts)

        future = model.make_future_dataframe(periods=forecast_horizon, freq='M')
        forecast = model.predict(future)

        # Pegando as previsões do horizonte futuro
        y_pred = forecast['yhat'][-forecast_horizon:].values
        y_true = ts['y'].values[-forecast_horizon:]

        rmse = root_mean_squared_error(y_true, y_pred)
        scores[col] = rmse
        forecasts[col] = y_pred

    return forecasts, scores

In [ ]:
forecasts, scores = prophet(df)

#### Save results and show general score

In [7]:
df_forecasts = pd.DataFrame(forecasts)
df_scores = pd.DataFrame(list(scores.items()), columns=['Série', 'RMSE'])

In [8]:
df_forecasts.to_excel('../Output/v6/previsoes_prophet.xlsx', index=False)
df_scores.to_excel('../Output/v6/scores_prophet.xlsx', index=False)

In [9]:
print(f'WRMSE = {wrmse(df_scores["RMSE"])}')

WRMSE = 0.10073893604682788
